In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 节省内存读文件
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    @param df:
    @return:
    """
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

### 用户表

In [3]:
train_user = reduce_mem_usage(pd.read_csv('../../input/train/train_user.csv'))
test_user = reduce_mem_usage(pd.read_csv('test2/test_user.csv'))

Memory usage of dataframe is 635152.00 MB
Memory usage after optimization is: 256580.00 MB
Decreased by 59.6%
Memory usage of dataframe is 58128.00 MB
Memory usage after optimization is: 47978.00 MB
Decreased by 17.5%


In [4]:
train_user.head()

,phone_no_m,city_name,county_name,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,绵阳,江油分公司,1,46.062500,45.000000,45.000,45.00000,45.00000,45.00000,45.00000,45.000000,0
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,德阳,旌阳分公司,1,79.000000,79.187500,79.125,79.31250,41.40625,34.09375,59.40625,60.000000,0
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,成都,金堂分公司,2,18.843750,24.453125,22.000,21.40625,22.15625,21.09375,61.31250,63.000000,0
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,成都,高新分公司,2,19.203125,29.656250,130.375,173.12500,19.00000,19.00000,19.00000,23.203125,0
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,德阳,旌阳分公司,1,50.000000,50.000000,50.000,50.00000,50.00000,50.00000,50.00000,50.000000,0


In [5]:
test_user.head()

,phone_no_m,city_name,county_name,idcard_cnt,arpu_202005
0,da8d08e497cdcdc5d01ad4651f60cffc425dc7c580912e...,天府新区,直管区,1,51.20
1,eecada1ec44482e4764bf6878ca319cbb437af04ae8afc...,内江,东兴区分公司,1,69.20
2,d7bda00771b29cca43a67eecc337576847e750e1a353be...,自贡,富顺县分公司,1,9.00
3,2b7b5af4a7077a329a8d5972541a9c0fd1a485233d7fec...,南充,南部分公司,2,\N
4,290bd1e12f2026b9994a03dbd3506b577588601a31f31a...,成都,金牛分公司,1,41.00


In [6]:
test_user['arpu_202005'].replace('\\N', np.nan, inplace=True)
test_user['arpu_202005'] = test_user['arpu_202005'].astype(float)

In [7]:
train_user.label.value_counts().plot.pie();

In [8]:
train_user['label'].mean()

0.321323288568621

In [9]:
train_user.shape, test_user.shape

((6106, 13), (1450, 5))

In [10]:
# 训练集只取最后一个月的数据
train_user['arpu_202005'] = train_user['arpu_202003']

train_user.drop(['arpu_201908', 'arpu_201909', 'arpu_201910', 'arpu_201911',
                 'arpu_201912', 'arpu_202001', 'arpu_202002', 'arpu_202003'],
                axis=1,
                inplace=True)

In [11]:
train_user.shape, test_user.shape

((6106, 6), (1450, 5))

In [12]:
train_user.nunique().sort_values(ascending=False)

phone_no_m     6106
arpu_202005    2200
county_name     182
city_name        23
idcard_cnt       11
label             2
dtype: int64

In [13]:
train_user.to_hdf('train_user.h5', 'df')
test_user.to_hdf('test_user.h5', 'df')

### 通话表

In [14]:
train_voc = reduce_mem_usage(pd.read_csv('../../input/train/train_voc.csv'))
test_voc = reduce_mem_usage(pd.read_csv('test2/test_voc.csv'))

Memory usage of dataframe is 320987648.00 MB
Memory usage after optimization is: 255787058.00 MB
Decreased by 20.3%
Memory usage of dataframe is 9477696.00 MB
Memory usage after optimization is: 7552565.00 MB
Decreased by 20.3%


In [15]:
# 只取最后一个月的数据
train_voc = train_voc[train_voc['start_datetime'] >= '2020-03-01 00:00:00']

In [16]:
train_voc.shape, test_voc.shape

((709144, 8), (148087, 8))

In [17]:
train_voc.nunique().sort_values(ascending=False)

start_datetime    561615
opposite_no_m     281103
imei_m              6025
phone_no_m          4823
call_dur            2653
county_name          193
city_name             22
calltype_id            3
dtype: int64

In [18]:
train_voc.to_hdf('train_voc.h5', 'df')
test_voc.to_hdf('test_voc.h5', 'df')

### 短信表

In [19]:
train_sms = reduce_mem_usage(pd.read_csv('../../input/train/train_sms.csv'))
test_sms = reduce_mem_usage(pd.read_csv('test2/test_sms.csv'))

Memory usage of dataframe is 219152416.00 MB
Memory usage after optimization is: 171212853.00 MB
Decreased by 21.9%
Memory usage of dataframe is 4308128.00 MB
Memory usage after optimization is: 3365753.00 MB
Decreased by 21.9%


In [20]:
train_sms = train_sms[train_sms['request_datetime'] >= '2020-03-01 00:00:00']

In [21]:
train_sms.shape, test_sms.shape

((910606, 4), (134625, 4))

In [22]:
train_sms.nunique().sort_values(ascending=False)

request_datetime    496383
opposite_no_m       104792
phone_no_m            4867
calltype_id              2
dtype: int64

In [23]:
test_sms.nunique().sort_values(ascending=False)

request_datetime    79160
opposite_no_m       17930
phone_no_m           1392
calltype_id             2
dtype: int64

In [24]:
train_sms.to_hdf('train_sms.h5', 'df')
test_sms.to_hdf('test_sms.h5', 'df')

### 应用表

In [25]:
train_app = reduce_mem_usage(pd.read_csv('../../input/train/train_app.csv'))
test_app = reduce_mem_usage(pd.read_csv('test2/test_app.csv'))

Memory usage of dataframe is 105075392.00 MB
Memory usage after optimization is: 91940984.00 MB
Decreased by 12.5%
Memory usage of dataframe is 3183008.00 MB
Memory usage after optimization is: 2387288.00 MB
Decreased by 25.0%


In [26]:
train_app = train_app[train_app['month_id'] == '2020-03']

In [27]:
train_app.shape, test_app.shape

((382647, 4), (99465, 4))

In [28]:
train_app.nunique().sort_values(ascending=False)

flow          281286
phone_no_m      4006
busi_name       3255
month_id           1
dtype: int64

In [29]:
test_app.nunique().sort_values(ascending=False)

flow          69898
busi_name      3075
phone_no_m     1093
month_id          1
dtype: int64

In [30]:
train_app.to_hdf('train_app.h5', 'df')
test_app.to_hdf('test_app.h5', 'df')